In [1]:
import mido
from typing import List, Tuple
import random
import numpy as np

In [2]:
major_scale = (0, 2, 4, 5, 7, 9, 11) # 2 2 1 2 2 2 1
minor_scale = (0, 2, 3, 5, 7, 8, 10) # 2 1 2 2 1 2 2
note_names = ("C", "CD", "D", "DE", "E", "F","FG","G","GA","A","AB","B",) # tuple for identifying a notes by finding a remainder by dividing by 12 in order to remove octave dependance. We divide by 12 because we have 12 notes in one octave
scales = {}
offset_for_chords = 48

for number, note_name in enumerate(note_names): #create scales for every key in both major and minor
    scales[note_name] = [ (number+step)%12 for step in major_scale] # all major
    scales[note_name+"m"] = [ (number+step)%12 for step in minor_scale] #all minor

class KeyIdentifier:

    melody_notes = None
    melody = None
    key_freq = {}

    def __init__(self) -> None:

        for number, note_name in enumerate(note_names):

            self.key_freq[note_name] = 0 #number of notes in melody that coincide with scale of given key
            self.key_freq[note_name + "m"] = 0 #number of notes in melody that coincide with scale of given key


    def setMelody(self, melody : mido.MidiTrack):

        self.melody = melody # save original song
        self.melody_notes = [message.note % 12 for message in self.melody if message.type=="note_on"] # got all notes
        for key in self.key_freq: # set to 0 all coincides
            self.key_freq[key] = 0

    def determineKey(self) -> int: # number of note in scale from 0 to 11

        for note in self.melody_notes: # Firstly calculate two the most appropriate keys according to coincidence of notes to keys scales
            for key, value in scales.items(): # value is a list of notes in the scale this key
                if note in value: # if current melody note in scale of this key then
                    self.key_freq[key] = self.key_freq[key] + 1 # +1 point to key if we have coincided note.
        total_max = 0
        rel_keys = [] # here we will have two relative keys
        for key, value in keyident.key_freq.items(): # find max value in dict
            if value > total_max:
                total_max = value

        for key in keyident.key_freq.keys(): #find keys with max value
            if keyident.key_freq[key] is total_max:
                rel_keys.append(key)

        last_note = (self.melody_notes[-1]) % 12

        # Check in this order: I -> V -> IV -> III
        for val in [0,4,3,2]:
            for key in rel_keys:
                if scales[key][val] is last_note:
                    return key
        # next code called if last note says almost nothing
        first_note = (self.melody_notes[0]) % 12
        for val in [0,4,3,2]:
            for key in rel_keys:
                if scales[key][val] is first_note:
                    return key

In [3]:
class ChordBuilder:
    major_steps = (0,4,7)
    minor_steps = (0,3,7)
    sus2_steps = (0,2,7)
    sus4_steps = (0,5,7)
    dim_steps = (0,3,6)

    def __init__(self, key : str) -> None:
        self.scale = scales[key]
        self.isMajor = True if not key.endswith("m") else False # to properly create major/minor chords

    def createMajorChord(self, first_note) ->  List[int]:
        return [first_note + step for step in ChordBuilder.major_steps]

    def createMinorChord(self, first_note) ->  List[int]:
        return [first_note + step for step in ChordBuilder.minor_steps]

    def createSUS2Chord(self, first_note) ->  List[int]:
        return [first_note + step for step in ChordBuilder.sus2_steps]

    def createSUS4Chord(self, first_note) ->  List[int]:
        return [first_note + step for step in ChordBuilder.sus4_steps]

    def createDIMChord(self, first_note) -> List[int]:
        return [first_note + step for step in ChordBuilder.dim_steps]

    def getPool(self) -> List[List[int]]:
        # Firstly create all minor and major chords
        pool = []

        triad_major = (self.createMajorChord, self.createMinorChord, self.createMinorChord, self.createMajorChord, self.createMajorChord, self.createMinorChord, self.createDIMChord)

        triad_minor = (self.createMinorChord, self.createDIMChord, self.createMajorChord, self.createMinorChord, self.createMinorChord, self.createMajorChord, self.createMajorChord)

        # all chords are created according to tables from assignment description
        if self.isMajor:
            for idx, func in enumerate(triad_major):
                pool.append(func(self.scale[idx]))
            # sus2 and sus4 next
            for idx, note in enumerate(self.scale):
                if (idx != 2) and (idx != 6):
                    pool.append(self.createSUS2Chord(note))

            for idx, note in enumerate(self.scale):
                if (idx != 3) and (idx != 6):
                    pool.append(self.createSUS4Chord(note))
        else:
            for idx, func in enumerate(triad_minor):
                pool.append(func(self.scale[idx]))
            #sus2 and sus4 next
            for idx, note in enumerate(self.scale):
                if (idx != 1) and (idx != 4):
                    pool.append(self.createSUS2Chord(note))

            for idx, note in enumerate(self.scale):
                if (idx != 1) and (idx != 5):
                    pool.append(self.createSUS4Chord(note))

        return pool

In [4]:
class EvolutionaryAlgorithm:


    def __init__(self, melody : List[int],
                 elite : int, numOfIndividuals : int,
                 numOfGenes : int,
                 generationsNumber : int,
                 numOfMutations : int,
                 numOfMutationsInsideMutation : int,
                 numOfCrossovers : int,
                 numOfRandomGeneCrossovers : int) -> None:

        self.numOfGenerations = generationsNumber # how many iterations we will have
        self.numOfMutations = numOfMutations # how many individuals to mutate
        self.numOfCrossovers = numOfCrossovers # how many offspring to get using crossover
        self.numOfRandomGeneCrossovers = numOfRandomGeneCrossovers #how many random genes to crossover in one iteration
        self.numOfGenes = numOfGenes # how many genes we have in population
        self.numOfMutationsInsideMutation = numOfMutationsInsideMutation # how many chords to mutate
        self.numOfIndividuals = numOfIndividuals # how many individuals we have at once.
        self.melody = melody
        self.elite = elite


    def get_random_chord(self) -> List[int]: # one chord among 17 available
        return pool[random.randint(0, 17)]

    def get_random_individual(self) -> List[List[int]]: # one accompanement
        return [self.get_random_chord() for i in range(self.numOfGenes)]

    def get_random_population(self) -> List[List[List[int]]]: # a bunch of accompanements
        return [self.get_random_individual() for i in range(self.numOfIndividuals)]

    def get_chord_fitness(self, chord : List[int]) -> int:
        # 10 point for every note intersection among melody part and chord
        fitness = 0
        for note in self.melody:
            if note in chord:
                fitness = fitness + 10

        return fitness

    def get_individual_fitness(self, individual : List[List[int]]) -> int:
        return sum([self.get_chord_fitness(chord) for chord in individual])


    def crossover(self, population : List[List[List[int]]]) -> List[List[List[int]]]:
        """
        Do given by numOfCrossovers field number of crossovers of individuals
        Every crossover consist of replacing random number of genes in 1st parent by genes in 2nd parent
        Returns numOfCrossovers chords obtained using crossover inside population
        :rtype: List[List[int]]
        """
        answer = []
        for i in range(self.numOfCrossovers):
            #todo not population but generation
            first_parent = random.choice(population) # individual 1. list[list[]]
            second_parent = random.choice(population) # individual 2

            while first_parent == second_parent: # we are searching for non identical parents
                second_parent = random.choice(population)

            offspring = first_parent

            for k in range(self.numOfRandomGeneCrossovers):
                randNumOfGene = random.randrange(0, self.numOfGenes) # what chord to switch
                offspring[randNumOfGene] = second_parent[randNumOfGene] # switching

            answer.append(offspring)

        return answer

    def mutation(self, population : List[List[List[int]]]) -> List[List[List[int]]]:
        for i in range(self.numOfMutations):
            idx = random.randrange(self.numOfIndividuals) # random individual index
            for k in range(self.numOfMutationsInsideMutation):
                randomGene = random.randrange(0,self.numOfGenes) # choose random gene
                population[idx][randomGene] = pool[random.randrange(0, 17)] # mutate this random gene

        return population

    def selection(self, population : List[List[List[int]]], offsprings : List[List[List[int]]]) -> List[List[List[int]]]:
        answer = population

        population_fitness = [self.get_individual_fitness(individual) for individual in population]
        offsprings_fitness = [self.get_individual_fitness(individual) for individual in offsprings]

        indices_population_sorted = np.argsort(population_fitness)
        indices_offsprings_sorted = np.argsort(offsprings_fitness)

        sorted_population = []
        sorted_offsprings = []

        for index in indices_population_sorted:
            sorted_population.append(population[index])

        for index in indices_offsprings_sorted:
            sorted_offsprings.append(offsprings[index])

        answer[self.elite:] = sorted_population[-self.elite:]
        answer[-(self.numOfIndividuals - self.elite):] = sorted_offsprings[-(self.numOfIndividuals - self.elite)]

        return answer

    def evolution(self) -> List[List[int]]:
        pass

In [7]:
track = mido.MidiFile("input2.mid")
keyident = KeyIdentifier()
keyident.setMelody(track)
key = keyident.determineKey()
chordBuilder = ChordBuilder(key)
pool = chordBuilder.getPool() # we have only 17 available chord without taking into account inversed triads

accord_length = track.ticks_per_beat

fragments = [] #list of lists of notes. every list of notes belong to one chord

time = 0
fragment = [] # list of notes
for message in track.tracks[1]:
    if message.type == "note_on":
        time = time + message.time
        if message.time != 0: # means pause
            if time >= accord_length:
                fragments.append(fragment)
                fragment = []
                time = time - accord_length
        fragment.append(message.note)

    elif message.type == "note_off":
        time = time + message.time
        if time >= accord_length:
            fragments.append(fragment)
            fragment = []
            time = time - accord_length
            if time !=0: # one note for two fragments
                fragment.append(message.note)
if fragment:
    fragments.append(fragment)